# Introduction to geomedians

Geomedians are 

*see https://github.com/digitalearthafrica/deafrica-sandbox-notebooks/blob/master/Frequently_used_code/Generating_geomedian_composites.ipynb for useful material on defining geomedians. Probably need to simplify and condense the information. Aim: users understand why to use geomedian and how to use it*

*Would have been mentioned in the Time video from the previous section*

## `xarray` functions mentioned in trello

There is a trello card for Week 3: Explain xarray timeseries

Not sure where that best fits in, possibly in this section.

## Exercise: Create a geomedian

Note to self: Discuss usage of 'Exercise' vs not

### Load data (as before, using `load_ard()`)

### Plot RGB for all loaded data

### Make geomedian using the function

### Plot RGB of composite

## Optional activity

*Reference composites notebook to see other examples of composites (possibly [this notebook](https://github.com/digitalearthafrica/deafrica-sandbox-notebooks/blob/master/Frequently_used_code/Generating_composites.ipynb) or [this notebook](https://github.com/digitalearthafrica/deafrica-sandbox-notebooks/blob/master/Frequently_used_code/Generating_geomedian_composites.ipynb) have not investigated thoroughly)*

## Conclusion

Compare individually-plotted data vs RGB of composite &mdash; it should be better?